In [19]:
import re
from cloudmesh.common.Shell import Shell
from cloudmesh.common.util import readfile

notebook = Shell.map_filename("~/cm/cloudmesh_mllog.log").path

mynotebook = readfile(notebook)
list_of_mllogs = []

count_lines = 0
for line in mynotebook.splitlines():
    #line.replace('\\', '')
    if 'F=' in line and 'S=' in line and 'E=' in line and 'NNSE' in line:
        list_of_mllogs.append(line)
        count_lines += 1
        print(line)

print(count_lines)

:::MLLOG {"namespace": "cloudmesh", "time_ms": 1668027247664, "event_type": "POINT_IN_TIME", "key": "RunTFTCustomVersion DLResults Graph Title event_num:0", "value": "{'extrastring': 'Training 0.72242 11/09/2022, 15:54:02 UTC EARTHQ-newTFTv29 Length=1775, Location Summed Results Magnitude 2 weeks Now, AVG MSE F=0.003839 S=0.003048 E=0.00478 TOTAL summedErr/SummedObs 50.68% NNSE  A:0.481 AT:0.48 S:0.313 ST:0.3'}", "metadata": {"file": "/tmp/ipykernel_123144/3089554875.py", "lineno": 214}}
:::MLLOG {"namespace": "cloudmesh", "time_ms": 1668027247721, "event_type": "POINT_IN_TIME", "key": "RunTFTCustomVersion DLResults Graph Title event_num:1", "value": "{'extrastring': 'Validation 0.74745 11/09/2022, 15:54:02 UTC EARTHQ-newTFTv29 Length=1775, Location Summed Results Magnitude 2 weeks Now, AVG MSE F=0.004157 S=0.003404 E=0.005054 TOTAL summedErr/SummedObs 47.01% NNSE  A:0.481 AT:0.48 S:0.313 ST:0.3'}", "metadata": {"file": "/tmp/ipykernel_123144/3089554875.py", "lineno": 214}}
:::MLLOG {"

In [23]:
fixed_dicts = []
for mllog in list_of_mllogs:
    working_line = mllog.strip()
    # print(type(working_line))
    # print(working_line)
    working_line = working_line.replace(':::MLLOG ', '')
    # print('a', working_line)
    working_line = working_line[:-3]
    # print('b', working_line)
    working_line = working_line.replace('\\', '')
    # print('c', working_line)
    working_line = working_line.split(', "metadata":', 1)[0]
    working_line += '}'
    working_line = eval(working_line)
    working_line['value'] = eval(working_line['value'])
    # print('d', working_line)
    # print(type(working_line))

    # interpreting scientific values
    parse_line = working_line['value']['extrastring']
    parse_list = parse_line.split()

    # this function returns the value after a specified word
    def looking_for(seeking_value: str):
        found_it = False
        for word in parse_list:
            if found_it:
                return word
            if word == seeking_value:
                found_it = True
                continue

    validation_value = looking_for('Validation')
    if validation_value:
        working_line['value']['validation'] = validation_value
    training_value = looking_for('Training')
    if training_value:
        working_line['value']['training'] = training_value

    list_of_values = []
    working_line['value']['MSE'] = {}
    for word in parse_list:
        if '=' in word and ('F' in word or 'S' in word or 'E' in word):
            temp_word = word.split('=')
            working_line['value']['MSE'][temp_word[0]] = temp_word[1]
        elif 'A:' in word or 'AV:' in word or 'S:' in word or 'SV:' in word:
            temp_word = word.split(':')
            working_line['value'][temp_word[0]] = temp_word[1]
    working_line['value']['title'] = working_line['key']
    working_line['key'] = 'result'

    program_title = looking_for('UTC')
    working_line['value']['program'] = program_title
    nnse = looking_for('summedErr/SummedObs')
    working_line['value']['NNSE'] = nnse
    comment = looking_for('TOTAL')
    working_line['value']['comment'] = comment

    future = re.search('FUTURE (.*) MSE', working_line['value']['extrastring'])
    try:
        working_line['value']['future'] = future.group(1)
    except AttributeError:
        pass

    loc_sum = re.search('Location Summed Results (.*),', working_line['value']['extrastring'])
    working_line['value']['location_summed_results'] = loc_sum.group(1)
    del working_line['value']['extrastring']

    fixed_dicts.append(working_line)


In [24]:
from pprint import pprint
for fixed_dict in fixed_dicts:
    pprint(fixed_dict)
print(len(fixed_dicts))
    # print(working_line)


{'event_type': 'POINT_IN_TIME',
 'key': 'result',
 'namespace': 'cloudmesh',
 'time_ms': 1668027247664,
 'value': {'A': '0.481',
           'MSE': {'E': '0.00478', 'F': '0.003839', 'S': '0.003048'},
           'NNSE': '50.68%',
           'S': '0.313',
           'comment': 'summedErr/SummedObs',
           'location_summed_results': 'Magnitude 2 weeks Now',
           'program': 'EARTHQ-newTFTv29',
           'title': 'RunTFTCustomVersion DLResults Graph Title event_num:0',
           'training': '0.72242'}}
{'event_type': 'POINT_IN_TIME',
 'key': 'result',
 'namespace': 'cloudmesh',
 'time_ms': 1668027247721,
 'value': {'A': '0.481',
           'MSE': {'E': '0.005054', 'F': '0.004157', 'S': '0.003404'},
           'NNSE': '47.01%',
           'S': '0.313',
           'comment': 'summedErr/SummedObs',
           'location_summed_results': 'Magnitude 2 weeks Now',
           'program': 'EARTHQ-newTFTv29',
           'title': 'RunTFTCustomVersion DLResults Graph Title event_num:1',
    